In [1]:
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import preprocessor as p
import os
import time

#### Scraping data using tweepy+Twitter API:

In [2]:
# Twitter credentials
#Authentication
consumer_key = ''
consumer_secret = ''
access_key = ''
access_secret = ''

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

#### Searching for tweets via twitter query:

In [5]:
text_query = '$TSLA min_replies:0 min_faves:2 -filter:retweets'
max_tweets = 2500
 
# query for Twitter API
tweets = tweepy.Cursor(api.search,q=text_query).items(max_tweets)
 
# Pulling information from tweets iterable object
#Specifying columns:
tweets_list = [[tweet.text, tweet.created_at, tweet.id_str, tweet.user.name, tweet.user.screen_name, tweet.user.id_str, tweet.user.location, tweet.user.url, tweet.user.description] for tweet in tweets]
 
# Creation of dataframe from tweets_list
# Did not include column names to simplify code 
tweets_df = pd.DataFrame(tweets_list)

#### Retrieving tweets with proper place format:

In [14]:
tweets_with_loc = tweets_df.loc[tweets_df[6].str.contains(',',regex=False)]
tweets_df.loc[tweets_df[6].str.contains(',',regex=False)]

,0,1,2,3,4,5,6,7,8
16,RT @JustinPulitzer: 🆕Weekend Review VIDEO 11/8...,2020-11-07 11:25:23,1325036623915048960,Justin Pulitzer Ťг𝓐ⒹＥᔕ,JustinPulitzer,18944615,"New York, NY",https://t.co/RCFzoeEtpi,"Prop Trader blending Technical, Fundamental & ..."
23,RT @fly4dat: Problem solving skills: A+\n\n$TS...,2020-11-07 11:09:44,1325032682259181568,Gustavo Litovsky,agusnox,2996161037,"Dallas, TX",https://t.co/rYKZZ9e3P5,Wireless Tech Condottiere. Investor. $tslaq - ...
24,@saxena_puru I think I know what firm you’re t...,2020-11-07 11:07:44,1325032181039816705,Hedge Fund Harry,fund_harry,1081006211674394626,"Toronto, Canada",None,"Trading, neuroscience, and memes."
44,"RT @fly4dat: 🚨🚨🚨 Based on official data, #Tesl...",2020-11-07 10:41:39,1325025617155338240,Sharkeys Machine,MachineSharkeys,1071058340988493826,"Manchester, UK",https://t.co/05yHEk2yr0,Sat in amazement watching Central banks destro...
63,RT @jhall: Over here waiting for @lorakolodny ...,2020-11-07 10:19:20,1325020000583782402,⚡️ fourth row popcorn 🍿,4thRowPopcorn,1278839568775561216,"Los Angeles, CA",None,just enjoying the drama. my views are not mine
...,...,...,...,...,...,...,...,...,...
2484,@SJosephBurns If you have to ask the stock pri...,2020-11-06 14:34:59,1324721950107688961,𝓟𝓸𝓵𝓲𝓽𝓮𝓵𝔂 𝓒𝓸𝓻𝓻𝓮𝓬𝓽 💙,CorrectlyPolite,2538298702,"San Francisco, CA",None,"Seek improvement, but expect not perfection. C..."
2487,When gambling on the stock is really what it’s...,2020-11-06 14:34:41,1324721873955946497,Adam Gausepohl,AdamGausepohl,133461285,"Newport Beach, CA",https://t.co/gbjErMq2AV,"Entrepreneur and investor. CEO at @PopShorts, ..."
2489,Is this the right time to buy $Tsla ?,2020-11-06 14:33:53,1324721671140499456,Al-Ameen 👑 #Cybersec,ALOFFAWY,150315520,"Abuja, Nigeria",https://t.co/DVxnk7m8Yv,Offensive Security Professional.
2494,Hang in there $TSLA,2020-11-06 14:32:53,1324721419092172802,Toni Marconi 🦩🦩🦩,gatica1813,158262281,"Pennsylvania, USA",None,human\n#Twitterphilanthropy


#### Saving to .csv file 

In [17]:
tweets_with_loc.to_csv(r'C:\Users\DELL\Desktop\tweets_with_loc.csv', index = False)